In [1]:
from itertools import chain

import pandas as pd

from ddeserts.annotate import RACES
from ddeserts.annotate import add_all_stat_columns
from ddeserts.annotate import add_geo_columns
from ddeserts.annotate import with_columns_sorted
from ddeserts.load import load_cvap_data
from ddeserts.load import load_charter_cities

In [2]:
# load the data
pre_filter = lambda line: ('California' in line and 'CDP' not in line)

df = load_cvap_data('Place', pre_filter=pre_filter)
len(df)

485

In [3]:
# add geo columns and post-filter
add_geo_columns(df)
df = df[(df['state'] == 'California') & df['geotype'].isin(['city', 'town'])]
len(df)

482

In [4]:
# add charter city data
charter_cities = load_charter_cities()
df['geosubtype'] = 'general law'
df.loc[df['name'].isin(charter_cities), 'geosubtype'] = 'charter'

In [5]:
# add stat columns and sort
add_all_stat_columns(df)
df = with_columns_sorted(df)

In [6]:
# sort by common name of city (Ventura, not San Buenaventura)
df.sort_values(by=['name'], inplace=True)

In [7]:
df

geoid                        geoname   geosubtype  \
table line                                                                
Place 25130  16000US0600296      Adelanto city, California      charter   
      25156  16000US0600394  Agoura Hills city, California  general law   
      25221  16000US0600562       Alameda city, California      charter   
      25247  16000US0600674        Albany city, California      charter   
      25286  16000US0600884      Alhambra city, California      charter   
...                     ...                            ...          ...   
      44786  16000US0686930    Yountville city, California  general law   
      44799  16000US0686944         Yreka city, California  general law   
      44812  16000US0686972     Yuba City city, California  general law   
      44825  16000US0687042       Yucaipa city, California  general law   
      44838  16000US0687056  Yucca Valley town, California  general law   

            geotype          name       state  adu_est  adu_moe  cit_est  \
table line                                                                 
Place 25130    city      Adelanto  California    21995      653    29610   
      25156    city  Agoura Hills  California    16140      393    19320   
      25221    city       Alameda  California    62565      524    72155   
      25247    city        Albany  California    14665      241    16290   
      25286    city      Alhambra  California    70315      496    69830   
...             ...           ...         ...      ...      ...      ...   
      44786    city    Yountville  California     2760       81     2940   
      44799    city         Yreka  California     5660      180     7500   
      44812    city     Yuba City  California    49455      404    58480   
      44825    city       Yucaipa  California    40175      646    50875   
      44838    city  Yucca Valley  California    16730      502    21165   

             cit_moe  ...  prop_his_adu_dvap_est  prop_his_adu_dvap_moe  \
table line            ...                                                 
Place 25130      532  ...               0.265078               0.070049   
      25156      468  ...               0.243333               0.251120   
      25221      810  ...               0.152388               0.113577   
      25247      629  ...               0.281013               0.191676   
      25286      975  ...               0.149142               0.051519   
...              ...  ...                    ...                    ...   
      44786       41  ...               0.041667               0.480925   
      44799      107  ...               0.067308               0.395029   
      44812      771  ...               0.344544               0.050885   
      44825      470  ...               0.142425               0.109854   
      44838      199  ...               0.100149               0.195586   

             prop_ind_adu_dvap_est  prop_ind_adu_dvap_moe  \
table line                                                  
Place 25130               0.000000               2.121320   
      25156               0.000000              23.000000   
      25221               0.000000               0.699250   
      25247               0.000000               0.821591   
      25286               0.040816               0.661508   
...                            ...                    ...   
      44786               0.000000               1.979899   
      44799               0.000000               0.403313   
      44812               0.000000               0.442872   
      44825               0.062500               1.117735   
      44838               0.000000               0.754247   

             prop_pac_adu_dvap_est  prop_pac_adu_dvap_moe  \
table line                                                  
Place 25130               0.166667               2.091584   
      25156               0.000000              23.000000   
      25221               0.125000         

In [8]:
df.to_csv('output/ca-cities.csv')

In [ ]:
# general disenfranchisement data
cols = ['name', 'geosubtype'] + list(chain(*(
    [f'{pop}_est', f'{pop}_moe'] for pop in ('tot', 'adu', 'dvap', 'prop_adu_dvap'))))
dg = df[cols]
dg.to_csv('output/ca-cities-dvap.csv')

In [ ]:
# racial disparity data
cols = ['name', 'geosubtype'] + list(chain(*(
    [f'{pop}_est', f'{pop}_moe'] for pop in (
        ['tot', 'adu'] + 
        [f'prop_dvap_{r}' for r in RACES] +
        [f'racial_disp_{r}' for r in RACES] +
        [f'prop_adu_{r}' for r in RACES] +
        [f'prop_cvap_{r}' for r in RACES]
    ))))
dg = df[cols]
dg.to_csv('output/ca-cities-racial-disp.csv')

In [ ]:
dg = df[['name', 'geosubtype', 'racial_disp_his_est', 'racial_disp_his_moe']]
dg[dg['geosubtype'] == 'charter'].sort_values(by=['racial_disp_his_est'])[:30]

In [ ]:
dg = df[['name', 'geosubtype', 'racial_disp_asn_est', 'racial_disp_asn_moe']]
dg[dg['geosubtype'] == 'charter'].sort_values(by=['racial_disp_asn_est'])[:30]

In [ ]:
dg = df[['name', 'geosubtype', 'racial_disp_wht_est', 'racial_disp_wht_moe']]
dg[dg['geosubtype'] == 'charter'].sort_values(by=['racial_disp_wht_est'], ascending=False)[:30]

In [ ]:
dg = df[df['name'] == 'King City']
dg[['prop_adu_his_est', 'prop_adu_his_moe', 'prop_dvap_his_est', 'prop_dvap_his_moe', 
    'prop_adu_dvap_est', 'prop_adu_dvap_moe', 'prop_his_adu_dvap_est', 'prop_his_adu_dvap_moe', 
    'adu_est', 'cvap_est', 'his_adu_est', 'his_adu_moe', 'his_cvap_est', 'his_cvap_moe']]